In [141]:
#! python3
import xlsxwriter
from pathlib import Path
import pandas as pd
import numpy as np
from pandas import DataFrame
from datetime import datetime

class Data:
    def __init__(self):
        self.temp_av = []
        self.hum = []
        self.qq = []
        self.wind = []
        self.rows = []
        self.columns = []
        self.wb = []
        self.heat = []

    def reading(self):
        # reading excel file
        # directory below should contains only cleaned xlsx files
        p = Path('C:/Users/kinan/Desktop/Bioinformatyka/Modern Data Analytics/cleaned')
        files = list(p.glob('*.xlsx'))
        files.sort  # sort in alphabetical order
        self.hum = pd.read_excel(files[0]).drop(['FR'], axis=1)
        self.qq = pd.read_excel(files[1])
        self.temp_av = pd.read_excel(files[2]).drop(['FR'], axis=1)
        self.wind = pd.read_excel(files[3]).drop(['FR'], axis=1)
        self.rows = self.qq.iloc[:, 0]
        self.columns = list(self.qq.columns)

    def calculation(self):
        # formula: 0.735×Ta+0.0374×RH+0.00292×Ta×RH +7.619×SR−4.557×SR2−0.0572×WS−4.064
        for i in range(1, len(self.columns)):
            for j in range(0, len(self.rows)):
                sr = self.qq.iloc[j, i] * 0.001
                ws = self.wind.iloc[j, i] * 0.1
                ta = self.temp_av.iloc[j, i]
                rh = self.wind.iloc[j, i]
                self.wb.append(0.735*ta + 0.0374*rh + 0.00292*ta*rh + 7.619*sr - 4.557*(sr**2) - 0.0572*ws - 4.064)

    def write_to_excel(self):
        wb = xlsxwriter.Workbook('wet_bulb2.xlsx')
        sheet = wb.add_worksheet()
        cnt = 0

        for i in range(1, len(self.columns)):
            cnt_row = 0
            sheet.write(0, i, self.columns[i])
            for j in range(0, len(self.rows)):
                if 3 < int(self.rows[j][5:7]) < 10:
                    print(j+1, i, self.wb[cnt])
                    sheet.write(cnt_row+1, i, self.wb[cnt])
                    if i == 1:
                        sheet.write(cnt_row+1, 0, self.rows[j])
                    cnt_row += 1
                cnt += 1
        wb.close()

    def percentile_90(self):
        wb = pd.read_excel('wet_bulb_quarters.xlsx') # read the we_bulb file from current working directory

        de_90 = np.percentile(wb['DE'], 90)
        es_90 = np.percentile(wb['ES'], 90)
        nl_90 = np.percentile(wb['NL'], 90)
        per = [de_90, es_90, nl_90]  # percentiles list
        print(de_90, es_90, nl_90)

        for i in range(1, 4):
            cnt_h = 0  # number of heatwaves
            in_row = False
            to_break = 0  # counter for the days after heatwave
            start = []  # date of first day of heatwave
            stop = []  # dat of last day of heatwave
            first = True  # first day of heatwave

            for j in range (0, len(wb.iloc[:, i])):
                if wb.iloc[j, i] > per[i-1] and wb.iloc[j-1, i] > per[i-1]:
                    if first:
                        start.append(wb.iloc[j, 0])
                        first = False
                    in_row = True  # if True we are within heatwave
                    to_break = 0  # in heatwave, number after heatwave = 0
                else:
                    # counting days with temp below percentile value
                    if in_row:
                        to_break += 1

                    # if we have more then 7 days after high temperatures it is the end of heatwave
                    if to_break > 7:
                        to_break = 0
                        in_row = False  # end of heatwave
                        cnt_h += 1
                        first = True
                        stop.append(wb.iloc[j, 0])
                        #print("HEAT:"+ wb.iloc[j, 0])

            date_format = "%Y-%m-%d"
            days = []
            for i in range(0, cnt_h):                
                a = datetime.strptime(start[i], date_format)
                b = datetime.strptime(stop[i], date_format)
                delta = b - a
                days.append(delta.days) # that's it
            # print(days)
            period = [start, stop, days]
            df = DataFrame(period).transpose()
            df.columns = ['Firstday','LastDay', 'duration']
            # print(df)
            # print(cnt_h)
            # print(start)
            # print(stop)
            df.Firstday = pd.to_datetime(df.Firstday)
            df.set_index('Firstday', inplace=True)
            df_sum = df.resample('Q').duration.sum()
            # print(df_sum)
            df_sum.index = pd.DatetimeIndex(df_sum.index)
            df_summer = df_sum[(df_sum.index.quarter == 2) | (df_sum.index.quarter == 3)]
            print(df_summer)
            df_summer.to_excel('NumDay_quarter_{}.xlsx'.format(i), index=False)

wet_bulb = Data()
#wet_bulb.reading()
#wet_bulb.calculation()
#wet_bulb.write_to_excel()
wet_bulb.percentile_90()

15.25045341880328 19.202664194612755 15.78399920915981
Firstday
1996-06-30    13
1996-09-30     8
1997-06-30     0
1997-09-30    19
1998-06-30     8
1998-09-30    17
1999-06-30     0
1999-09-30    36
2000-06-30    17
2000-09-30     0
2001-06-30     0
2001-09-30    44
2002-06-30    10
2002-09-30    27
2003-06-30    25
2003-09-30    35
2004-06-30     0
2004-09-30    21
2005-06-30    22
2005-09-30    16
2006-06-30    49
2006-09-30     8
2007-06-30     9
2007-09-30    10
2008-06-30    17
2008-09-30    25
2009-06-30     0
2009-09-30    39
2010-06-30    39
2010-09-30     8
2011-06-30    17
2011-09-30     9
2012-06-30     8
2012-09-30    21
2013-06-30    10
2013-09-30    26
2014-06-30    10
2014-09-30    20
2015-06-30     8
2015-09-30    55
2016-06-30     8
2016-09-30    37
2017-06-30    30
2017-09-30    20
2018-06-30    10
2018-09-30    38
2019-06-30    28
2019-09-30    26
Name: duration, dtype: int64
Firstday
1996-09-30    12
1997-06-30     0
1997-09-30     8
1998-06-30     0
1998-09-30    